In [1]:
#-*-coding: utf-8
json_hierarchy = [
        {
            "div": "감사위원회",
            "child": "조사담당관"
#             /안전담당관/감사담당관"
        },
        {
            "div": "경제진흥본부",
            "child": {
                "div": "창조경제기획관",
                "child": "도시농업과"
#                 /경제정책과/민생경제과
#                 /화융합경제과/디지털창업과/투자유치과/소상공인지원과"
            }
        },
#         {
#             "div": "관광체육국",
#             "child": "관광정책과/체육진흥과/체육정책과/관광사업과"
#         },
        {
            "div": "기술심사담당관"
        },
#         {
#             "div": "기획조정실",
#             "child": [
#                 {
#                     "div": "국제교류담당관"
#                 },
#                 {
#                     "div": "정책기획관",
#                     "child": "기획담당관/법률지원담당관/대외협력담당관/조직담당관/법무담당관/평가담당관"
#                 },
#                 {
#                     "div": "해외도시협력담당관"
#                 },
#                 {
#                     "div": "재정기획관",
#                     "child": "예산담당관/시민참여예산과/공기업담당관/재정관리담당관/시민참여예산담당관"
#                 }
#             ]
#         }
    ]

In [2]:
from urllib import parse

# URL 추출
urls = [];
tail = '&dept%5B%5D='
def extract_value(data, origin_url):
  if (len(data)) > 2:
    for single_div in data :
      url = str(origin_url)
      if isinstance(single_div, dict):
        try:
          if isinstance(single_div['child'], str):
            #요기 최종
            ## pares 안해주면 한글 url이 깨진다요
            url += parse.quote(single_div['div'])
            url += tail
            for div in single_div['child'].split('/'):
              url_final = url + parse.quote(div)
              urls.append(url_final)
#               print(url_final)
              urls.append(url_final)
          else:
            url += parse.quote(single_div['div'])
            url += tail
            extract_value(single_div['child'], url)
        except KeyError:
#           url += parse.quote(single_div.get('div'))
#           urls.append(url)
          print('keyerr')
      else:
        print("no dict : ")
  else:
    url = str(origin_url)
    child_list = data.get('child')
    for child in child_list.split('/'):
      url_final = url + parse.quote(data.get('div')) + tail + parse.quote(child)
      urls.append(url_final)
#       print(url_final)
      
extract_value(json_hierarchy, "")
urls[:5]

keyerr


['%EA%B0%90%EC%82%AC%EC%9C%84%EC%9B%90%ED%9A%8C&dept%5B%5D=%EC%A1%B0%EC%82%AC%EB%8B%B4%EB%8B%B9%EA%B4%80',
 '%EA%B0%90%EC%82%AC%EC%9C%84%EC%9B%90%ED%9A%8C&dept%5B%5D=%EC%A1%B0%EC%82%AC%EB%8B%B4%EB%8B%B9%EA%B4%80',
 '%EA%B2%BD%EC%A0%9C%EC%A7%84%ED%9D%A5%EB%B3%B8%EB%B6%80&dept%5B%5D=%EC%B0%BD%EC%A1%B0%EA%B2%BD%EC%A0%9C%EA%B8%B0%ED%9A%8D%EA%B4%80&dept%5B%5D=%EB%8F%84%EC%8B%9C%EB%86%8D%EC%97%85%EA%B3%BC']

In [3]:
# 필요한 라이브러리 가져오기
from urllib import request
!pip install html5lib
import html5lib
!pip install lxml
import pandas as pd
import numpy as np

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [25]:
import re
# 2017년 1년치 데이터 크롤링
df = pd.DataFrame()
base_url_head = "https://opengov.seoul.go.kr/expense?dept%5B%5D=%EC%84%9C%EC%9A%B8%EC%8B%9C%EB%B3%B8%EC%B2%AD&dept%5B%5D="
base_url_tail = "&year=2017&month="
column_div = "1소속부서"

# url돌면서 url 만들기
for div in urls:
  div_url = (base_url_head + div + base_url_tail)
  unquote_div = parse.unquote(div)
  div_name = re.search('(.*)&', unquote_div).group(1)
  for month in range(1,4): 
    url_month = div_url + str(month)
    try:
      df_month = pd.read_html(url_month)[1]
      df_month[column_div] = div_name
      df = df.append(df_month)
    except IndexError:
      print('err', url_month)


#   mayor_12months = mayor_12months.append(df)

#   ## 1 category 칼럼 넣기
#   ## 년/ 월 / 일 / 시간 분리하기
#   ## 레스토랑명 / 지역 분리하기

err https://opengov.seoul.go.kr/expense?dept%5B%5D=%EC%84%9C%EC%9A%B8%EC%8B%9C%EB%B3%B8%EC%B2%AD&dept%5B%5D=%EA%B2%BD%EC%A0%9C%EC%A7%84%ED%9D%A5%EB%B3%B8%EB%B6%80&dept%5B%5D=%EC%B0%BD%EC%A1%B0%EA%B2%BD%EC%A0%9C%EA%B8%B0%ED%9A%8D%EA%B4%80&dept%5B%5D=%EB%8F%84%EC%8B%9C%EB%86%8D%EC%97%85%EA%B3%BC&year=2017&month=3


데이터가 없는 항목도 많다. 크롤링 하는데 엄청 오래걸리네... 엑셀에 넣어둬야겠당

In [14]:
df.describe()

,연번,집행금액
count,150.000000,150.000000
mean,11.580000,137937.200000
std,7.267903,125367.285501
min,1.000000,8100.000000
25%,5.000000,47850.000000
50%,11.000000,96000.000000
75%,17.000000,184525.000000
max,28.000000,490000.000000


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150 entries, 0 to 6
Data columns (total 9 columns):
연번       150 non-null int64
부서명      150 non-null object
집행일시     150 non-null object
집행장소     150 non-null object
집행목적     150 non-null object
대상인원     150 non-null object
결제방법     150 non-null object
집행금액     150 non-null int64
1소속부서    150 non-null object
dtypes: int64(2), object(7)
memory usage: 11.7+ KB


In [16]:
df.shape

(150, 9)

In [23]:
del df['연번']
del df['부서명']

KeyError: '연번'

In [ ]:
df.head(2)

In [20]:
df.tail(2)

,연번,부서명,집행일시,집행장소,집행목적,대상인원,결제방법,집행금액,1소속부서
5,6,경제진흥본부 도시농업과,2017-02-23 13:02,우도일식 (중구 태평로1가 지하1층),도시농업 민간단체 공모사업추진간담회,"도시농업과장, 도시농업지원팀장 등 4명",카드,60000,경제진흥본부&dept[]=창조경제기획관
6,7,경제진흥본부 도시농업과,2017-02-28 16:47,장군보쌈 (중구 무교로 19),직원 인사이동에 따른 직원격려 간담회,도시농업과장 및 도시농업과직원 27명,카드,470000,경제진흥본부&dept[]=창조경제기획관
